# Wikipedia Bot for Biography Generation

In [1]:
!pip install transformers
!pip install pywikibot
!wget -nc -q <DOWNLOAD_WIKI_user-config.py>
!wget -nc -q <DOWNLOAD_WIKI_user-password.py>
!python -m spacy download en_core_web_lg

import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from nltk.translate import bleu_score, gleu_score
import nltk
from nltk.corpus import stopwords
import pywikibot
import spacy

UsageError: Cell magic `%%shell` not found.


In [ ]:
# Load resources
tokenizer = GPT2Tokenizer.from_pretrained("mbien/fdh-wikibio")
model = GPT2LMHeadModel.from_pretrained("mbien/fdh-wikibio")
model = model.to("cuda:0")
nltk.download("stopwords")
nlp = spacy.load('en_core_web_lg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Structural accuracy scores

In [ ]:
def structural_accuracy(sample):
  parts = sample.split("<|summary|>")
  keywords = re.sub(r"(<\|[^\|]+\|>)|( nan )", "", parts[0])
  keywords = re.sub(r"\(|\)|-|,", " ", keywords)
  keywords = re.sub(r"(\s{2,})", " ", keywords)
  keywords = re.sub(r"(^\s+)|(\s+$)", "", keywords).lower().split(" ")
  keywords = [keyword for keyword in keywords if keyword not in stopwords.words('english')]
  generated = parts[1].lower()
  matches = [keyword for keyword in keywords if keyword in generated]

  return len(matches)/len(keywords)

# Bot implementation

In [ ]:
df_generate = pd.read_csv("data/dataset_no_en_full.csv").drop("summary", axis=1)

# If person has no item description (Q[0-9]+), use fname and lname to fix it whenever possible
df_generate.loc[df_generate.itemLabel.str.startswith("Q"), "itemLabel"] = df_generate.loc[df_generate.itemLabel.str.startswith("Q"), "fnames"].astype(str).str.replace("^nan$","")+" "+df_generate.loc[df_generate.itemLabel.str.startswith("Q"), "lnames"].astype(str).str.replace("^nan$", "")
df_generate.itemLabel = df_generate.itemLabel.str.replace("(^\s+)|(\s+$)", "")
df_generate = df_generate[df_generate.itemLabel!=""]
df_generate

,item,itemLabel,fnames,lnames,itemDescription,birth,death,professions
0,Q18945651,Marsilio Cagnati,NaN,NaN,1543-1612,1543,1612,physician
1,Q55901911,Angelo Gualandris,Angelo,NaN,botanist (1750-1788),1750,1788,"naturalist, geologist, agronomist, botanist"
2,Q1522146,Giacomo Nani,Giacomo,Nani,politician (1725-1797),1725,1797,politician
3,Q21656610,Joannes Bernardus Felicianus,Giovanni,NaN,NaN,1490,1552,physician
4,Q19838679,Ermolao Donà,Ermolao,NaN,NaN,NaN,1450,"politician, diplomat"
...,...,...,...,...,...,...,...,...
1615,Q16531271,Bartolomeo Orioli,Bartolomeo,NaN,Italian painter (1568-1627),1568,1627,painter
1616,Q22916128,Paolo Calvi,Paolo,Calvi,Italian biographer (1716-1781),1716,1781,biographer
1618,Q3852178,Matteo Polo,"Maffeo, Matteo",Polo,Italian merchant (1252-1309),1252,1309,merchant
1619,Q3903724,Pietro Buratti,Pietro,Buratti,Italian poet,1773,1832,poet


In [ ]:
# Last adjustment before actual generation: try to find the images for these people in wiki commons
import requests
import csv
import urllib.parse

def find_commons_image(ids):
  query = "SELECT ?item ?pic WHERE { ?item wdt:P31 wd:Q5; wdt:P27 wd:Q4948. ?item wdt:P18 ?pic}"
  result = requests.get("https://query.wikidata.org/sparql", params={'query': query}, headers={'Accept': 'text/csv'}).text
  data = list(csv.reader(result.split("\r\n"), delimiter=',', quotechar='"'))[:-1]
  df = pd.DataFrame(data[1:], columns=data[0])
  return df

images = find_commons_image(("wd:"+df_generate.item).values)
images.pic = images.pic.str.replace("http://commons.wikimedia.org/wiki/Special:FilePath/", "")
images.pic = images.pic.apply(urllib.parse.unquote)
images.item = images.item.str.replace("http://www.wikidata.org/entity/", "")
df_generate = df_generate.merge(images, on='item', how='left')
df_generate[df_generate.pic.notna()]

,item,itemLabel,fnames,lnames,itemDescription,birth,death,professions,pic
10,Q3880819,Oddo degli Oddi,Oddo,NaN,Italian physician,1478,1558,physician,"Oddo degli Oddi. Line engraving, 1688. Wellcom..."
12,Q43288825,Camillo Rospigliosi,Camillo,Rospigliosi,NaN,1714,1763,NaN,Camillo rospigliosi di agostino massucci (1737...
20,Q15654345,Tommaso Formenton,Tommaso,Formenton,Italian architect,1440,1492,architect,Tommaso Formenton- Palazzo Thiene.jpg
30,Q18507796,Antonio Bosa,Antonio,NaN,Italian sculptor (1780-1845),1780,1845,sculptor,Malatesta Antonio Bosa.jpg
34,Q16559788,Giovanni Battista Cromer,"Giovanni, Giovanni Battista",NaN,Italian painter (1665-1745),1665,1745,painter,San Giacomo dall'Orio (Venice) - Immacolata co...
...,...,...,...,...,...,...,...,...,...
1528,Q18945373,Giorgio Summaripa,Giorgio,NaN,1435-1497,1435,1497,poet,"Summaripa - Martirio di Simone da Trento, circ..."
1534,Q16571672,Giovanni Battista Ponchino,Giambattista,NaN,Italian painter (1500-1570),1500,1570,painter,Giovanni Battista Ponchino - Mercurio e Minerv...
1541,Q51156,Domenico Maria Federici,Domenico,NaN,historic Italian,1739,1808,historian,"Federici, Domenico Maria - Commentario sopra l..."
1550,Q4023585,Zaccaria Bricito,Zaccaria,NaN,NaN,1802,1851,Catholic priest,Zaccaria Bricito.jpg


In [ ]:
# Define generation function

def generate_biography(model, tokenizer, person_definition):
  prompt = person_definition
  end_token = tokenizer.encode("<|end|>")[0]
  pad_token = tokenizer.encode("<|pad|>")[0]

  encoded_prompt = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
  encoded_prompt = encoded_prompt.to("cuda:0")

  outputs = []

  while len(outputs) == 0:
    outputs_enc = model.generate(encoded_prompt, do_sample=True, min_length=128, max_length=1024, device=0, 
                            top_k=10, top_p=0.97, num_return_sequences=20,
                            eos_token_id=end_token, pad_token_id=pad_token)

    outputs = [tokenizer.decode(output_enc) for output_enc in outputs_enc if (len(output_enc)<1024) or (output_enc[-1] in [end_token, pad_token])]
    if len(outputs) == 0:
      print("Outputs too long! Repeating...")
      #outputs = ['<|section|>'.join(tokenizer.decode(output_enc).split("<|section|>")[:-1]) for output_enc in outputs_enc]
      #print([tokenizer.decode(output_enc) for output_enc in outputs_enc])

  # We don't care about end tokens, we may want to add another section afterwards!
  outputs=[output.replace("<|end|>", "").replace("<|pad|>", "") for output in outputs]

  return outputs

def data2text(row):
  out = '<|start|> '
  out += row.itemLabel
  if str(row.itemDescription) != "nan":
    out += ' <|description|> '
    out += str(row.itemDescription)
  if str(row.professions) != "nan":
    out += f' <|professions|> {row.professions}'
  if str(row.birth)!="nan":
    out += f' <|birth|> {row.birth}'
  if str(row.death)!="nan":
    out += f' <|death|> {row.death}'
  
  return out + ' <|summary|>'


# {{Infobox person
# | name          = AAA
# | image         = blah.jpg
# | honorific_suffix       = BBB
# | birth_date    = 1750
# | death_date    = 1788
# | nationality   = [[wikipedia:Republic of Venice|Republic of Venice]]
# | occupation    = DDD
# }}


def data2infobox(row):
  out = '{{Infobox person\n'
  out += '| name = ' + row.itemLabel + '\n'
  if str(row.pic) != "nan":
    out += '| image = ' + row.pic + '\n'
  if str(row.itemDescription) != "nan":
    out += '| honorific_suffix = ' + row.itemDescription + '\n'
  if str(row.professions) != "nan":
    out += '| occupation = ' + row.professions + '\n'
  if str(row.birth)!="nan":
    out += '| birth_date = ' + row.birth + '\n'
  if str(row.death)!="nan":
    out += '| death_date = ' + row.death + '\n'
  
  out += "| nationality = [[wikipedia:Republic of Venice|]]\n"
  out += "| website = {{URL|https://www.wikidata.org/wiki/" + row["item"] + "|" + row["item"] + "}}\n"
  return out + '}}\n'



def generate_valid_wikipage(model, tokenizer, person):
  acceptance_rate = 1.0
  best_rate = 0.0
  best_bio = ""
  while best_rate < acceptance_rate:
    # Every time decrease by a percent
    acceptance_rate -= 0.05
    print(f"Generating for acceptance rate {acceptance_rate}, best score {best_rate}")
    bios = generate_biography(model, tokenizer, person)

    for bio in bios:
      # Drop if duplicate section
      sections = re.findall("== ([A-Za-z]+) ==", bio)
      if len(sections) != len(set(sections)):
        print("Section duplicate detected")
        continue

      if bool(re.search("(<\|[a-z]+\|>)|(\[[^\]]+\])", bio.split("<|summary|>")[1].replace("<|section|>", ""))):
        print("Wild tokens found!")
        continue

      score = structural_accuracy(bio)
      if score > best_rate:
        best_bio = bio
        best_rate = score
  return best_bio

In [ ]:
site = pywikibot.Site()
site.login()

def enrich_valid_wikipage(text):
  results = nlp(text)
  replacables = []
  for ent in results.ents:
    if ent.label_ == "GPE":
      replacables.append(ent.text)
    if (ent.label_ == "ORG") and ("university" in ent.text.lower()):
      replacables.append(ent.text)

  if len(replacables)==0:
    return text

  replacables = [re.sub("(^the\s+)|(\(.+)", "", rep).strip() for rep in replacables]

  out = ""
  for rep in replacables:
    splits = text.split(rep, 1)
    out += splits[0] + f"[[wikipedia:{rep}|]]"
    text = splits[1]
  return out + text

def upload_wikibio(person, infobox):
  person_data, person_wiki = person.split(" <|summary|> ")[0:2]
  person_name = re.findall("<\|start\|>\s*([^<]+)\s*<", person_data)[0]
  page = pywikibot.Page(site, person_name)
  page.text = infobox + enrich_valid_wikipage(person_wiki)
  page.save("New version of biography generated")
  return page.text

def generation_pipeline(row):
  content = data2text(row)
  infobox = data2infobox(row)

  bio = generate_valid_wikipage(model, tokenizer, content).replace(" <|section|> ", "\n\n")
  final_page = upload_wikibio(bio, infobox)
  #print(final_page)

tqdm.pandas()

In [ ]:
df_generate[1088:].progress_apply(generation_pipeline, axis=1)

Generating for acceptance rate 0.95, best score 0.0
Wild tokens found!
Section duplicate detected


Page [[Gian Girolamo Zannichelli]] saved


Generating for acceptance rate 0.95, best score 0.0
Wild tokens found!
Wild tokens found!
Wild tokens found!


Page [[Camillo Silvestri]] saved


Generating for acceptance rate 0.95, best score 0.0


...

Generating for acceptance rate 0.95, best score 0.0


Page [[Pietro Buratti]] saved


Generating for acceptance rate 0.95, best score 0.0
Wild tokens found!
Wild tokens found!



Page [[Lodovico Fogliano]] saved


1088    None
1089    None
1090    None
1091    None
1092    None
        ... 
1562    None
1563    None
1564    None
1565    None
1566    None
Length: 479, dtype: object

In [ ]:
def source_patch(row):
  infobox = data2infobox(row)

  page = pywikibot.Page(site, row.itemLabel)

  bio = page.text
  bio = bio.split("}}\n")[-1]

  page.text = infobox + bio
  page.save("Biography patched with wikidata")

df_generate.progress_apply(source_patch, axis=1)

Page [[Marsilio Cagnati]] saved
Sleeping for 8.0 seconds, 2020-12-10 12:24:25
Page [[Angelo Gualandris]] saved
Sleeping for 7.9 seconds, 2020-12-10 12:24:35
Page [[Giacomo Nani]] saved
Sleeping for 8.0 seconds, 2020-12-10 12:24:45
Page [[Joannes Bernardus Felicianus]] saved
Sleeping for 7.7 seconds, 2020-12-10 12:24:55
Page [[Ermolao Donà]] saved
Sleeping for 7.9 seconds, 2020-12-10 12:25:05
Page [[Antonio Maria Traversi]] saved
Sleeping for 8.0 seconds, 2020-12-10 12:25:15
Page [[Bernardino Vitali]] saved
Sleeping for 8.0 seconds, 2020-12-10 12:25:25
Page [[Fina Buzzaccarini]] saved
Sleeping for 8.0 seconds, 2020-12-10 12:25:35
Page [[Giuseppe Rocco Volpi]] saved
Sleeping for 8.0 seconds, 2020-12-10 12:25:45
Page [[Ottavio Padovanino]] saved
Sleeping for 7.6 seconds, 2020-12-10 12:25:55
Page [[Oddo degli Oddi]] saved
Sleeping for 6.6 seconds, 2020-12-10 12:26:06
Page [[Onofrio Paganini]] saved
Sleeping for 7.6 seconds, 2020-12-10 12:26:15
Page [[Camillo Rospigliosi]] saved
Sleeping fo

Page [[Lodovico Fogliano]] saved


0       None
1       None
2       None
3       None
4       None
        ... 
1562    None
1563    None
1564    None
1565    None
1566    None
Length: 1567, dtype: object